In [1]:
import torch 
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader, Data
import h5py
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"
import hdf5storage

#no_measurements = 1200 

data_path = './' # os.getcwd() 
#datasetR = [] 



In [2]:
no_measurements = 16
datasetTrain = [] 
filename = data_path+'data/bootstrapped_train_9.h5' 
for item in range(no_measurements): 
    x = torch.tensor(hdf5storage.read(path='/data/'+str(item+1)+'/x', filename=filename, options=None), dtype=torch.float)
    xt = torch.tensor(hdf5storage.read(path='/data/'+str(item+1)+'/xt', filename=filename, options=None), dtype=torch.float)
    o = torch.tensor(hdf5storage.read(path='/data/'+str(item+1)+'/o', filename=filename, options=None), dtype=torch.float)
 #   onode = torch.tensor(hdf5storage.read(path='/data/'+str(item+1)+'/onode', filename=filename, options=None), dtype=torch.float)
 #   omarker = torch.tensor(hdf5storage.read(path='/data/'+str(item+1)+'/omarker', filename=filename, options=None), dtype=torch.float)
    edge_index = torch.tensor(hdf5storage.read(path='/data/'+str(item+1)+'/edge_index', filename=filename, options=None), dtype=torch.long)
    edge_attr = torch.tensor(hdf5storage.read(path='/data/'+str(item+1)+'/edge_feature', filename=filename, options=None), dtype=torch.float)
    y = torch.tensor(hdf5storage.read(path='/data/'+str(item+1)+'/y', filename=filename, options=None), dtype=torch.float)
    datasetTrain.append(Data(x=x, xt=xt, o=o, y=y, edge_index=edge_index.t().contiguous(), edge_attr=edge_attr)) 
    print([item, datasetTrain[item]])

[0, Data(edge_attr=[183638, 4], edge_index=[2, 183638], o=[183638, 1], x=[577, 4], xt=[577, 4], y=[577, 4])]
[1, Data(edge_attr=[37506, 4], edge_index=[2, 37506], o=[37506, 1], x=[227, 4], xt=[227, 4], y=[227, 4])]
[2, Data(edge_attr=[77814, 4], edge_index=[2, 77814], o=[77814, 1], x=[677, 4], xt=[677, 4], y=[677, 4])]
[3, Data(edge_attr=[40784, 4], edge_index=[2, 40784], o=[40784, 1], x=[341, 4], xt=[341, 4], y=[341, 4])]
[4, Data(edge_attr=[99942, 4], edge_index=[2, 99942], o=[99942, 1], x=[450, 4], xt=[450, 4], y=[450, 4])]
[5, Data(edge_attr=[37608, 4], edge_index=[2, 37608], o=[37608, 1], x=[332, 4], xt=[332, 4], y=[332, 4])]
[6, Data(edge_attr=[191090, 4], edge_index=[2, 191090], o=[191090, 1], x=[553, 4], xt=[553, 4], y=[553, 4])]
[7, Data(edge_attr=[46590, 4], edge_index=[2, 46590], o=[46590, 1], x=[338, 4], xt=[338, 4], y=[338, 4])]
[8, Data(edge_attr=[126274, 4], edge_index=[2, 126274], o=[126274, 1], x=[1084, 4], xt=[1084, 4], y=[1084, 4])]
[9, Data(edge_attr=[43230, 4], edg

In [3]:
no_measurements = 1 
datasetTest = [] 

filename = data_path+'data/bootstrapped_test_9.h5' 
#filename = data_path+'data/gt_graph_random_large_outliers_real_updated.h5' 
for item in range(no_measurements): 
    x = torch.tensor(hdf5storage.read(path='/data/'+str(item+1)+'/x', filename=filename, options=None), dtype=torch.float)
    xt = torch.tensor(hdf5storage.read(path='/data/'+str(item+1)+'/xt', filename=filename, options=None), dtype=torch.float)
    o = torch.tensor(hdf5storage.read(path='/data/'+str(item+1)+'/o', filename=filename, options=None), dtype=torch.float)
 #   onode = torch.tensor(hdf5storage.read(path='/data/'+str(item+1)+'/onode', filename=filename, options=None), dtype=torch.float)
 #   omarker = torch.tensor(hdf5storage.read(path='/data/'+str(item+1)+'/omarker', filename=filename, options=None), dtype=torch.float)
    edge_index = torch.tensor(hdf5storage.read(path='/data/'+str(item+1)+'/edge_index', filename=filename, options=None), dtype=torch.long)
    edge_attr = torch.tensor(hdf5storage.read(path='/data/'+str(item+1)+'/edge_feature', filename=filename, options=None), dtype=torch.float)
    y = torch.tensor(hdf5storage.read(path='/data/'+str(item+1)+'/y', filename=filename, options=None), dtype=torch.float)
    datasetTest.append(Data(x=x, xt=xt, o=o.t(), y=y, edge_index=edge_index.t().contiguous(), edge_attr=edge_attr)) 
    print([item, datasetTest[item]])

[0, Data(edge_attr=[535814, 4], edge_index=[2, 535814], o=[1, 535814], x=[2152, 4], xt=[2152, 4], y=[2152, 4])]


In [4]:
def qmul(q, r):
    """
    Multiply quaternion(s) q with quaternion(s) r.
    """
    assert q.shape[-1] == 4
    assert r.shape[-1] == 4

    original_shape = q.shape

    # Compute outer product
    terms = torch.bmm(r.view(-1, 4, 1), q.view(-1, 1, 4))

    w = terms[:, 0, 0] - terms[:, 1, 1] - terms[:, 2, 2] - terms[:, 3, 3]
    x = terms[:, 0, 1] + terms[:, 1, 0] - terms[:, 2, 3] + terms[:, 3, 2]
    y = terms[:, 0, 2] + terms[:, 1, 3] + terms[:, 2, 0] - terms[:, 3, 1]
    z = terms[:, 0, 3] - terms[:, 1, 2] + terms[:, 2, 1] + terms[:, 3, 0]
    return torch.stack((w, x, y, z), dim=1).view(original_shape)

def inv_q(q):
    """
    Inverse quaternion(s) q .
    """
    assert q.shape[-1] == 4
    original_shape = q.shape
    return torch.stack((q[:, 0], -q[:, 1], -q[:, 2], -q[:, 3]), dim=1).view(original_shape)

In [5]:
from torch.nn import Sequential as Seq, Linear, ReLU, BatchNorm1d as BN, Dropout
from torch_geometric.nn import MessagePassing

class EdgeConvRot(MessagePassing):
    def __init__(self, in_channels, edge_channels, out_channels):
        super(EdgeConvRot, self).__init__(aggr='mean', flow="target_to_source") #  "Max" aggregation.
        self.mlp = Seq(Linear(2*in_channels+edge_channels, out_channels),
               ReLU(),
               Linear(out_channels, out_channels))
            
    def forward(self, x, edge_index, edge_attr):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x, edge_attr=edge_attr)

    def message(self, x_i, x_j, edge_attr): 
        W = torch.cat([torch.cat([x_i, x_j], dim=1), edge_attr], dim=1)  # tmp has shape [E, 2 * in_channels]
        W = self.mlp(W) 
        return W
            
    def propagate(self, edge_index, size, x, edge_attr):    
        row, col = edge_index
        x_i = x[row]
        x_j = x[col]
        i, j = (0, 1) if self.flow == 'target_to_source' else (1, 0) 
        edge_out = self.message(x_i, x_j, edge_attr)
        out = scatter_(self.aggr, edge_out, edge_index[i], dim_size=size[i])
        return out, edge_out 


In [6]:
import torch
import torch.nn.functional as F
from torch_geometric.utils import scatter_
from torch_geometric.nn import GATConv
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax

def node_model(x, batch):
   # print(batch.shape)
    out, inverse_indices = torch.unique_consecutive(batch, return_inverse=True)
    quat_vals = x[inverse_indices] 
    q_ij = qmul(x, inv_q(quat_vals[batch]))  
    return q_ij 

def edge_model(x, edge_index):
    row, col = edge_index
    q_ij = qmul(x[col], inv_q(x[row]))  
    return q_ij 

def MLP(channels, batch_norm=True):
    return Seq(*[
        Seq(Linear(channels[i - 1], channels[i]), ReLU())
        for i in range(1, len(channels))
    ])
class EdgePred(torch.nn.Module):
    def __init__(self, in_channels, edge_channels):
        super(EdgePred, self).__init__()
        self.mlp = Seq(Linear(2*in_channels+edge_channels, 8),
                       ReLU(),
                       Linear(8, 1)) 
    def forward(self, xn, edge_index, edge_attr): 
        row, col = edge_index
        xn = torch.cat([xn[row], xn[col], edge_attr], dim=1)
        xn = self.mlp(xn) 
        return torch.sigmoid(xn) 
    
class GlobalSAModule(torch.nn.Module):
    def __init__(self, nn1, nn2):
        super(GlobalSAModule, self).__init__()
        self.nn1 = nn1
        self.nn2 = nn2

    def forward(self, x, batch): 
        xn = self.nn1(x)
      #  xn = F._max_pool1d(xn, x.size(1))
       # xn = scatter_('mean', xn, batch)
       # xn = xn[batch]  
        xn = torch.cat([xn, x], dim=1) 
     #   print(xn.shape)
      #  x = xn.unsqueeze(0).repeat(x.size(0), 1, 1) 
      #  batch = torch.arange(x.size(0), device=batch.device)
        return self.nn2(xn)
    
 

In [7]:
def update_attr(x, edge_index, edge_attr):
    row, col = edge_index
    x_i = x[row]
    x_j = inv_q(x[col])
 #   print(x_i.shape)
  #  print(x_j.shape)
   # print(edge_attr.shape) 
    W=qmul(edge_attr, x_i) 
    W=qmul(x_j, W) 
    return W 



def smooth_l1_loss(input, beta=0.05, size_average=True):
    """
    very similar to the smooth_l1_loss from pytorch, but with
    the extra beta parameter
    """
    nn0 = torch.min(1.0 - input[:, 0], 1.0 + input[:, 0])     
    
    n = torch.abs(nn0) + torch.sum(torch.abs(input[:, 1:]), dim=1)      
 #   cond = n < 5*beta
  #  n = torch.where(cond, n, 0*n)
    cond = n < beta
    loss = torch.where(cond, 0.5 * n ** 2 / beta, n - 0.5 * beta)
    if size_average:
        return loss.mean()
    return loss.sum()

def my_smooth_l1_loss(input, beta, alpha=0.05):
    """
    very similar to the smooth_l1_loss from pytorch, but with
    the extra beta parameter
    """
    nn0 = torch.min(1.0 - input[:, 0], 1.0 + input[:, 0]) 
    nn = torch.abs(nn0) + torch.sum(torch.abs(input[:, 1:]), dim=1) 
    beta = torch.squeeze(beta) 
    nn = torch.mul(nn, beta) 
   # print([nn.shape, beta.shape])
   # nn = nn + alpha*(torch.ones(beta.shape, dtype=torch.float, device=input.device) - beta) 
    cond = nn < alpha
    loss = torch.where(cond, 0.5 * nn ** 2 / alpha, nn - 0.5 * alpha)
    return loss.mean()

def my_smooth_l1_loss_new(input, beta, edge_index, size, alpha=0.05):
    """
    very similar to the smooth_l1_loss from pytorch, but with
    the extra beta parameter
    """
    nn0 = torch.min(1.0 - input[:, 0], 1.0 + input[:, 0]) 
    nn = torch.abs(nn0) + torch.sum(torch.abs(input[:, 1:]), dim=1) 
    beta = torch.squeeze(beta) 
    nn = torch.mul(nn, beta) 
    
    cond = nn < alpha
    loss = torch.where(cond, 0.5 * nn ** 2 / alpha, nn - 0.5 * alpha)   
    loss = scatter_('mean', loss, edge_index[0], dim_size=size(0)) 
        

    return loss.mean()

class Net(torch.nn.Module):
    def __init__(self): 
        super(Net, self).__init__() 
        self.no_features = 32   # More features for large dataset 
        self.conv1 = EdgeConvRot(4, 4, self.no_features) 
        self.conv2 = EdgeConvRot(self.no_features, self.no_features+4, self.no_features)  
        self.conv3 = EdgeConvRot(2*self.no_features, 2*self.no_features, self.no_features) 
        self.conv4 = EdgeConvRot(2*self.no_features, 2*self.no_features, self.no_features) 

        self.lin1 = Linear(self.no_features, 4) 
        
        self.m = torch.nn.Sigmoid() 
    def forward(self, data):
        x_org, edge_index, edge_attr, batch, beta = data.x, data.edge_index, data.edge_attr, data.batch, data.o  
        
        edge_attr_mod = update_attr(x_org, edge_index, edge_attr[:, :4])
        x1, edge_x1 = self.conv1(x_org, edge_index,edge_attr_mod)
        x1 = F.relu(x1)
        edge_x1 = F.relu(edge_x1)
        
        x2, edge_x2 = self.conv2(x1, edge_index, torch.cat([edge_attr_mod, edge_x1], dim=1))
        x2 = F.relu(x2)
        edge_x2 = F.relu(edge_x2)

        x3, edge_x3 = self.conv3(torch.cat([x2, x1], dim=1), edge_index, torch.cat([edge_x2, edge_x1], dim=1))
        x3 = F.relu(x3)
        edge_x3 = F.relu(edge_x3)
        
        x4, edge_x4 = self.conv4(torch.cat([x3, x2], dim=1), edge_index, torch.cat([edge_x3, edge_x2], dim=1))
        x4 = F.relu(x4)
        
        x = self.lin1(x4) 
       
        x = x + x_org #qmul(x, x_org) 
        x = F.normalize(x, p=2, dim=1) 
        
   #     loss1 = inv_q(edge_model(data.y, edge_index)) - edge_model(x, edge_index) 
        loss1 = qmul(inv_q(edge_model(data.y, edge_index)), edge_model(x, edge_index)) 
        loss1 = F.normalize(loss1, p=2, dim=1) 
#        loss1 = qmul(inv_q(edge_attr[:, :4]), edge_model(x, edge_index)) 
        loss1 = my_smooth_l1_loss_new(loss1[:, 0:], beta, edge_index, x_org.size)
        return x, loss1, beta   # node_model(x, batch),

In [8]:
PATH = 'checkpoint/finenet_9.pth' 
import time
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
training_exmpl = 0.8
model = Net().to(device) 
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
#checkpoint = torch.load(PATH)
#model.load_state_dict(checkpoint['model_state_dict'])
#optimizer.load_state_dict(checkpoint['optimizer_state_dict']) 
for g in optimizer.param_groups:
    g['lr'] = 0.00005

In [9]:
PATH = 'checkpoint/finenet_9.pth' 
import numpy as np 

train_loader = DataLoader(datasetTrain, batch_size=1, shuffle=True,num_workers=2)
test_loader = DataLoader(datasetTest, batch_size=1, shuffle=False,num_workers=2)

no_training = len(train_loader) 
no_testing = len(test_loader) 

model.train()
best_loss = 200 
t = time.time() 
count = 0 
val = 14688000
for epoch in range(2500):
    total_loss1 = 0 
    total_loss2 = 0 
    theta = [] 
    for indx, data in enumerate(train_loader):
        data_gpu = data.to(device)
        optimizer.zero_grad()
     #   print(data_gpu.y.shape)

        out, loss1, beta = model(data_gpu)
    #    out = data_gpu.x + out  
   #     out = qmul(data_gpu.x, out) 
      #  out = F.normalize(out, p=2, dim=1) 
     #   print(out.shape)
      #  loss1 = qmul(inv_q(data_gpu.edge_attr), edge_out)  
      #  out = node_model(out, data_gpu.batch)
      #  out = F.normalize(out, p=2, dim=1)
        out = qmul(inv_q(data_gpu.y), out) 
      #  loss1 = F.normalize(loss1, p=2, dim=1) 
        out = F.normalize(out, p=2, dim=1) 

    # F.smooth_l1_loss(10*edge_out, 10*data_gpu.edge_attr, size_average=None)  
      #  loss2 = F.mse_loss(out, data_gpu.y, size_average=None) 
   #    loss1 = loss1[:, 1:].abs().sum() #torch.nn.MSELoss(out, data_gpu.y)
        loss2 = smooth_l1_loss(out[:, 0:]) 
 
      #  loss2 = out[:, 1:].pow(2).sum() #(out - data_gpu.y).pow(2).sum() #torch.nn.MSELoss(out, data_gpu.y)
        loss = loss1 + 0.25*loss2 
        loss.backward()
        optimizer.step()
      #  print([indx, loss.item()])
     #   time.sleep(0.05)
        count = count + 1
        if epoch % 2 == 0:
            total_loss1 = total_loss1 + loss1.item() 
            total_loss2 = total_loss2 + loss2.item()
          #  out = qmul(inv_q(data_gpu.y), data_gpu.xt) 
            val2 = out.data.cpu().numpy()
            theta = np.concatenate((theta, 2.0*np.arccos(np.abs(val2[:, 0]))*180.0/np.pi ))
    total_loss1 = total_loss1/no_training
    total_loss2 = total_loss2/no_training
    if epoch % 10 == 0:
        total_loss1 = 0 
        total_loss2 = 0 
        for data in test_loader: 
            data_gpu = data.to(device)
            out, loss1, beta = model(data_gpu)
            out = qmul(inv_q(data_gpu.y), out) 
            out = F.normalize(out, p=2, dim=1) 
            loss2 = smooth_l1_loss(out[:, 0:]) 
            total_loss1 = total_loss1 + loss1.item() 
            total_loss2 = total_loss2 + loss2.item()
          #  out = qmul(inv_q(data_gpu.y), data_gpu.xt) 
            val2 = out.data.cpu().numpy()
            theta = np.concatenate((theta, 2.0*np.arccos(np.abs(val2[:, 0]))*180.0/np.pi ))
        # print([count, total_loss1/no_training, total_loss2/no_training], time.time() - t) 
        total_loss1 = total_loss1/no_testing
        total_loss2 = total_loss2/no_testing
    if epoch % 2 == 0: 
        print([epoch, "{0:.6f}".format(total_loss1), "{0:.2f}".format(np.mean(theta)), "{0:.2f}".format(np.median(theta)), "{0:.6f}".format(total_loss2)], "{0:.3f}".format(time.time() - t))
        if val > total_loss1/no_training : 
            val = total_loss1/no_training 
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                }, PATH) 


[0, '0.122196', '61.83', '33.48', '0.726958'] 1.449
[2, '0.098784', '61.20', '30.28', '0.579243'] 3.710
[4, '0.093762', '58.82', '26.68', '0.532862'] 5.997
[6, '0.089965', '57.06', '23.98', '0.493109'] 8.282
[8, '0.087147', '55.50', '21.53', '0.461195'] 10.538
[10, '0.110305', '52.83', '19.72', '0.565737'] 12.981
[12, '0.082976', '52.79', '17.45', '0.403192'] 15.223
[14, '0.081247', '51.41', '15.40', '0.369531'] 17.481
[16, '0.079695', '49.91', '13.38', '0.334547'] 19.812
[18, '0.078124', '48.65', '11.85', '0.307270'] 22.053
[20, '0.105208', '46.81', '11.88', '0.468718'] 24.467
[22, '0.076075', '48.10', '11.28', '0.295941'] 26.768
[24, '0.075517', '48.08', '11.33', '0.295234'] 29.028
[26, '0.075176', '48.10', '11.31', '0.295380'] 31.277
[28, '0.074939', '48.13', '11.32', '0.295520'] 33.544
[30, '0.105342', '46.84', '12.12', '0.474749'] 36.026
[32, '0.074674', '48.15', '11.29', '0.295858'] 38.256
[34, '0.074633', '48.14', '11.33', '0.295610'] 40.495
[36, '0.074548', '48.15', '11.34', '0

[298, '0.051672', '47.29', '9.59', '0.279395'] 342.906
[300, '0.079686', '45.70', '10.28', '0.433278'] 345.366
[302, '0.051469', '47.21', '9.60', '0.277975'] 347.629
[304, '0.051417', '47.31', '9.58', '0.279773'] 349.862
[306, '0.051328', '47.23', '9.58', '0.278295'] 352.121
[308, '0.051403', '47.30', '9.66', '0.279696'] 354.402
[310, '0.079050', '45.71', '10.28', '0.432170'] 356.814
[312, '0.051107', '47.22', '9.57', '0.278507'] 359.030
[314, '0.051054', '47.22', '9.78', '0.278264'] 361.197
[316, '0.051018', '47.21', '9.50', '0.277941'] 363.467
[318, '0.050938', '47.27', '9.63', '0.278631'] 365.693
[320, '0.078709', '45.64', '10.16', '0.432233'] 367.999
[322, '0.050765', '47.19', '9.48', '0.277256'] 370.257
[324, '0.050737', '47.23', '9.59', '0.278192'] 372.473
[326, '0.050828', '47.06', '9.34', '0.276825'] 374.727
[328, '0.050756', '47.42', '9.58', '0.280361'] 377.025
[330, '0.078548', '45.66', '10.02', '0.431881'] 379.476
[332, '0.050668', '47.27', '9.58', '0.278406'] 381.636
[334, 

[596, '0.045596', '46.64', '8.76', '0.269874'] 683.767
[598, '0.045465', '46.63', '8.84', '0.269468'] 686.056
[600, '0.073405', '45.06', '9.45', '0.417618'] 688.548
[602, '0.045423', '46.67', '8.72', '0.270780'] 690.854
[604, '0.045319', '46.67', '8.72', '0.270420'] 693.118
[606, '0.045185', '46.59', '8.76', '0.268919'] 695.369
[608, '0.045131', '46.58', '8.70', '0.268916'] 697.640
[610, '0.072979', '44.89', '9.37', '0.417470'] 700.111
[612, '0.045039', '46.55', '8.92', '0.269467'] 702.368
[614, '0.044835', '46.54', '8.67', '0.268580'] 704.629
[616, '0.044800', '46.57', '8.85', '0.269226'] 706.922
[618, '0.044836', '46.60', '8.71', '0.268786'] 709.149
[620, '0.072637', '45.07', '9.66', '0.416797'] 711.592
[622, '0.044623', '46.56', '8.79', '0.269148'] 713.872
[624, '0.044627', '46.54', '8.75', '0.268919'] 716.137
[626, '0.044411', '46.70', '8.74', '0.270391'] 718.336
[628, '0.044334', '46.41', '8.69', '0.267264'] 720.569
[630, '0.072104', '44.83', '9.17', '0.412892'] 723.039
[632, '0.0

[894, '0.038799', '45.76', '8.27', '0.259024'] 1026.404
[896, '0.038952', '45.78', '8.24', '0.260501'] 1028.628
[898, '0.038755', '45.67', '7.90', '0.258454'] 1030.859
[900, '0.067314', '44.00', '9.31', '0.397339'] 1033.300
[902, '0.038752', '45.67', '8.11', '0.258297'] 1035.529
[904, '0.038780', '45.91', '8.79', '0.261747'] 1037.804
[906, '0.038832', '45.78', '7.95', '0.260045'] 1040.135
[908, '0.038692', '45.64', '7.67', '0.257060'] 1042.433
[910, '0.067213', '44.05', '8.58', '0.397827'] 1044.910
[912, '0.038678', '45.67', '8.52', '0.258486'] 1047.153
[914, '0.038853', '45.76', '8.11', '0.259384'] 1049.433
[916, '0.038557', '45.91', '7.90', '0.259654'] 1051.668
[918, '0.038688', '45.82', '8.50', '0.260813'] 1053.955
[920, '0.067174', '44.06', '8.57', '0.400507'] 1056.407
[922, '0.038755', '45.78', '8.31', '0.259571'] 1058.632
[924, '0.038772', '45.83', '8.00', '0.260435'] 1060.897
[926, '0.038560', '45.52', '7.62', '0.255275'] 1063.178
[928, '0.038613', '45.78', '8.04', '0.259075'] 1

[1184, '0.037943', '45.17', '7.86', '0.251674'] 1359.025
[1186, '0.037906', '45.18', '7.26', '0.251855'] 1361.236
[1188, '0.037864', '45.16', '7.55', '0.252206'] 1363.541
[1190, '0.066569', '43.71', '8.38', '0.400529'] 1366.001
[1192, '0.037912', '45.10', '7.81', '0.251458'] 1368.262
[1194, '0.037922', '45.13', '7.67', '0.252035'] 1370.507
[1196, '0.038055', '45.15', '7.86', '0.252166'] 1372.812
[1198, '0.037833', '45.12', '7.63', '0.252156'] 1375.072
[1200, '0.066319', '43.67', '8.30', '0.401451'] 1377.496
[1202, '0.037801', '45.18', '7.30', '0.252394'] 1379.811
[1204, '0.038015', '45.08', '8.33', '0.252485'] 1382.090
[1206, '0.037792', '45.21', '8.05', '0.252626'] 1384.278
[1208, '0.037880', '45.08', '7.57', '0.252109'] 1386.450
[1210, '0.066024', '43.47', '8.12', '0.393296'] 1388.943
[1212, '0.038039', '45.10', '7.59', '0.250558'] 1391.123
[1214, '0.037896', '45.15', '7.71', '0.252565'] 1393.363
[1216, '0.037779', '45.18', '7.66', '0.251724'] 1395.613
[1218, '0.038068', '45.33', '7.

[1472, '0.037235', '44.68', '7.72', '0.246915'] 1688.909
[1474, '0.037280', '44.81', '8.55', '0.250755'] 1691.147
[1476, '0.037128', '44.76', '7.39', '0.247520'] 1693.446
[1478, '0.037130', '44.75', '7.32', '0.247307'] 1695.719
[1480, '0.065599', '43.14', '8.25', '0.391182'] 1698.145
[1482, '0.037269', '45.06', '7.30', '0.251000'] 1700.383
[1484, '0.037183', '44.75', '7.03', '0.246375'] 1702.551
[1486, '0.037196', '44.70', '7.16', '0.247105'] 1704.766
[1488, '0.037088', '44.84', '7.33', '0.247636'] 1707.049
[1490, '0.065372', '43.06', '8.26', '0.384178'] 1709.505
[1492, '0.037409', '44.94', '6.90', '0.249586'] 1711.769
[1494, '0.037116', '44.83', '7.08', '0.248100'] 1714.019
[1496, '0.037194', '44.75', '7.49', '0.246998'] 1716.263
[1498, '0.037093', '44.78', '7.16', '0.246926'] 1718.464
[1500, '0.065301', '42.89', '8.11', '0.382363'] 1720.943
[1502, '0.037171', '44.81', '7.31', '0.247176'] 1723.195
[1504, '0.037119', '44.76', '6.96', '0.246613'] 1725.443
[1506, '0.037056', '44.72', '7.

[1760, '0.065457', '42.90', '8.11', '0.387610'] 2018.311
[1762, '0.036460', '44.83', '7.42', '0.249134'] 2020.537
[1764, '0.036685', '44.48', '8.24', '0.245996'] 2022.801
[1766, '0.036562', '44.66', '6.82', '0.245765'] 2025.042
[1768, '0.036525', '44.48', '6.73', '0.245067'] 2027.262
[1770, '0.065270', '42.90', '8.20', '0.393095'] 2029.705
[1772, '0.036653', '44.44', '6.99', '0.243958'] 2031.946
[1774, '0.036403', '44.42', '6.92', '0.243572'] 2034.244
[1776, '0.036605', '44.72', '7.24', '0.245386'] 2036.508
[1778, '0.036527', '44.61', '6.96', '0.243288'] 2038.724
[1780, '0.065357', '42.84', '8.09', '0.392298'] 2041.126
[1782, '0.036562', '44.33', '7.25', '0.243190'] 2043.358
[1784, '0.036519', '44.32', '7.48', '0.243343'] 2045.658
[1786, '0.036399', '44.56', '7.15', '0.245449'] 2047.844
[1788, '0.036448', '44.44', '7.16', '0.244655'] 2050.139
[1790, '0.064905', '42.67', '7.90', '0.381668'] 2052.566
[1792, '0.036474', '44.41', '6.80', '0.243760'] 2054.734
[1794, '0.036847', '44.59', '7.

[2048, '0.035840', '44.14', '6.76', '0.239699'] 2347.610
[2050, '0.064447', '42.49', '7.36', '0.378206'] 2350.112
[2052, '0.036082', '44.19', '7.26', '0.242944'] 2352.355
[2054, '0.035701', '44.17', '6.84', '0.241314'] 2354.588
[2056, '0.035813', '44.08', '6.64', '0.239431'] 2356.829
[2058, '0.036353', '44.18', '8.17', '0.244013'] 2359.050
[2060, '0.064705', '42.47', '7.95', '0.382346'] 2361.474
[2062, '0.035812', '44.18', '6.42', '0.240578'] 2363.745
[2064, '0.036253', '44.13', '8.39', '0.244035'] 2366.057
[2066, '0.035976', '44.53', '7.05', '0.245015'] 2368.358
[2068, '0.035913', '43.80', '7.72', '0.237992'] 2370.610
[2070, '0.065004', '42.78', '7.93', '0.382045'] 2373.066
[2072, '0.035952', '44.26', '7.48', '0.242816'] 2375.315
[2074, '0.035935', '44.55', '7.05', '0.244715'] 2377.578
[2076, '0.035867', '44.63', '7.12', '0.245116'] 2379.812
[2078, '0.035946', '43.92', '7.61', '0.238480'] 2382.077
[2080, '0.064456', '42.72', '7.83', '0.379813'] 2384.515
[2082, '0.035928', '44.21', '7.

[2336, '0.035321', '43.66', '6.88', '0.235221'] 2677.305
[2338, '0.035164', '43.76', '6.51', '0.235762'] 2679.536
[2340, '0.065307', '42.60', '8.93', '0.404162'] 2681.990
[2342, '0.035233', '43.63', '6.82', '0.235113'] 2684.230
[2344, '0.035477', '43.85', '7.21', '0.239113'] 2686.494
[2346, '0.035343', '44.17', '6.51', '0.239439'] 2688.740
[2348, '0.035032', '43.76', '6.53', '0.235960'] 2691.010
[2350, '0.064235', '42.19', '7.36', '0.378525'] 2693.506
[2352, '0.035244', '43.76', '6.41', '0.235754'] 2695.712
[2354, '0.035494', '44.04', '6.70', '0.241814'] 2697.939
[2356, '0.035081', '43.78', '7.08', '0.236513'] 2700.156
[2358, '0.035316', '43.64', '6.42', '0.235407'] 2702.457
[2360, '0.064389', '42.29', '7.63', '0.384013'] 2704.904
[2362, '0.035264', '43.98', '6.56', '0.238283'] 2707.131
[2364, '0.035447', '43.87', '7.21', '0.238591'] 2709.352
[2366, '0.035322', '43.76', '6.45', '0.235638'] 2711.609
[2368, '0.035322', '43.58', '6.69', '0.233293'] 2713.839
[2370, '0.064600', '42.32', '7.

In [10]:
import numpy as np 
import math 
import h5py
import torch 
import time 
data_path = './' # os.getcwd() 

PATH = 'checkpoint/finenet_9.pth' 

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
model = Net().to(device) 
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])

test_loader = DataLoader(datasetTest, batch_size=1, shuffle=False)
#model = best_model 
#print(best_loss)
#pred_rot = []
model.eval()
total_loss = 0 
t = time.time() 
count = 0 
hf = h5py.File(data_path+'data/neurora_9.h5', 'w')
#hf = h5py.File(data_path+'data/pred_synthetic_rot_another.h5', 'w')
theta = [] 

for data in test_loader: 
    data_gpu = data.to(device)
    pred, out, beta = model(data_gpu)
   # data_gpu.x = pred
   # pred, out, beta = model(data_gpu)
   # data_gpu.x = pred
  #  pred, out, beta = model(data_gpu)
  #  pred = qmul(data_gpu.x, pred) 
  #  pred = data_gpu.x + pred
   # pred = F.normalize(pred, p=2, dim=1) 
  #  pred = node_model(pred, data_gpu.batch)
    pred = F.normalize(pred, p=2, dim=1)
        
    out = qmul(inv_q(data_gpu.y), pred) 
  #  out = qmul(inv_q(data_gpu.y), data_gpu.xt) 
    out = F.normalize(out, p=2, dim=1) 
    
    val2 = out.data.cpu().numpy()
  #  print(val.size)
    theta = np.concatenate((theta, 2.0*np.arccos(np.abs(val2[:, 0]))*180.0/np.pi )) 
    
    loss = out[:, 1:].pow(2).sum() 
 #   loss = (pred - data_gpu.y).pow(2).sum() 
    total_loss = total_loss + loss.item() 
    pred_rot = torch.cat([data_gpu.x, data_gpu.xt, pred, data_gpu.y], dim=1).data.cpu().numpy()
    hf.create_dataset('/data/'+str(count+1), data=pred_rot)
    count = count + 1 
   # print([len(pred_rot), (time.time()-t)/len(pred_rot)])
print(len(test_loader))
hf.close()
print([np.mean(theta), np.median(theta), (time.time() - t)/(test_loader.batch_size*len(test_loader))])


1
[32.13338178016772, 15.474712371826172, 2.222583770751953]
